In [9]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)


In [10]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, N = read_VSP_label()

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [11]:
label_visiprog = np.zeros((N,))

count = 0
for g in groups_VSP:
    label = False
    
    # check whether any entry in the group has been labelled yet
    # if yes, then merge with that existing label
    for i in g:
        if label_visiprog[i] != 0:
            label = label_visiprog[i]
            break
    
    # if no, then create a new label
    if label == False:
        count += 1
        label = count
        
    for i in g:
        label_visiprog[i] = label


In [12]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)

In [14]:
accuracy_STSIMM = []
accuracy_mat = []
accuracy_vsp_original = []
accuracy_vsp = []

for i in range(10):
    X_train, X_test, X_STSIMM_train, X_STSIMM_test, Y_mat_train, Y_mat_test, Y_VSP_train, Y_VSP_test = \
    train_test_split(raw_feature, X_STSIMM, label_material, label_visiprog, \
                     test_size=0.2, stratify=label_material)

    res = leave_one_sample_out(X_STSIMM_train, Y_mat_train, X_STSIMM_test, Y_mat_test)
    accuracy_STSIMM.append(res['accuracy'])

    model = LFDA()
    try:
        model.fit(X_train, Y_mat_train)
        X_mat_test = model.transform(X_test)
        res = leave_one_sample_out(X_mat_test, Y_mat_test)
        accuracy_mat.append(res['accuracy'])
    except Exception as e:
        acuracy_mat.append(NaN)

    model_vsp_original = LFDA()
    try:
        model_vsp_original.fit(X_train, Y_VSP_train)
        X_vsp_original_test = model_vsp_original.transform(X_test)
        res_vsp_original = leave_one_sample_out(X_vsp_original_test, Y_mat_test)
        accuracy_vsp_original.append(res_vsp_original['accuracy'])
    except Exception as e:
        accuracy_vsp_original.append(NaN)

    model_vsp = LFDA_VISIPROG()
    try:
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = leave_one_sample_out(X_vsp_test, Y_mat_test)
        accuracy_vsp.append(res_vsp['accuracy'])
    except Exception as e:
        accuracy_vsp.append(NaN)

INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on same data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on same data
I

In [15]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy_mat, 'STSIMM':accuracy_STSIMM, \
       'visiprog_original':accuracy_vsp_original}, columns=['STSIMM','material','visiprog_original','visiprog'])
df.loc['average'] = df.mean()
display(df)

,STSIMM,material,visiprog_original,visiprog
0,0.818147,0.980175,0.940145,0.977888
1,0.816241,0.979413,0.902021,0.972551
2,0.821578,0.979413,0.953488,0.974457
3,0.815478,0.977125,0.910789,0.972169
4,0.819672,0.974457,0.941289,0.969882
5,0.805947,0.980175,0.943957,0.974838
6,0.809760,0.976744,0.929089,0.972551
7,0.815478,0.979032,0.924895,0.973313
8,0.821197,0.978650,0.939382,0.971407
9,0.816241,0.975600,0.939382,0.972169


In [ ]:
print(Y_mat_test)